# Complete Unit 4 - “Agent Quality”:
- Listen to the summary podcast episode for this unit - https://www.youtube.com/watch?v=LFQRy-Ci-lk
- To complement the podcast, read the Agent Quality whitepaper - https://www.kaggle.com/whitepaper-agent-quality
- Complete these codelabs on Kaggle:
- A. Implement observability to help you debug your agents - https://www.kaggle.com/code/kaggle5daysofai/day-4a-agent-observability.
- B. Evaluate your agents - https://www.kaggle.com/code/kaggle5daysofai/day-4b-agent-evaluation.

### This whitepaper addresses the challenge of assuring quality in Al agents by introducing a holistic evaluation framework. The necessary technical foundation for this is Observability, built on three pillars: Logs (the diary), Traces (the narrative), and Metrics (the health report), enabling a continuous feedback loop using scalable methods like LLM-as-a-Judge and Human-in-the-Loop (HITL) evaluation.
- For today's codelabs, you'll learn how to use logs, traces, and metrics to get full visibility into your agent's decision-making process, allowing you to debug failures and understand why your agent behaves the way it does. In the second codelab, you'll learn how to evaluate your agents to score your agent's response quality and tool usage.

#### Find a complete list of scheduled livestreams and past recordings here - https://www.youtube.com/playlist?list=PLqFaTIg4myu9r7uRoNfbJhHUbLp-1t1YE.
- Tomorrow’s livestream (https://www.youtube.com/live/JW1Yybfxyr4) will be hosted by Kanchana Patlolla and Anant Nawalgaria at 11 AM PT / 8 PM CET / 12:30 AM IST (next day). They will be joined by codelabs author Sita Lakshmi - - Sangameswaran (https://www.linkedin.com/in/sitalakshmi04/), along with guests from Google: Wafae Bakkali, Turan Bulmus and Sian Gooding, and external guest Jiwei Liu from NVIDIA (https://www.nvidia.com/).
- This central resource page (https://www.kaggle.com/learn-guide/5-day-agents) contains all event materials, including assignments and important updates. Assignments are provided for your learning, so assignment submissions are not required, and you can complete them at your own pace.
- We want this community to be positive and supportive. Please follow Kaggle’s community guidelines found here (https://www.kaggle.com/community-guidelines).
- Be sure to ask all your questions about the podcast, readings, and codelabs in the ⁠5dgai-question-forum channel on Discord. You'll get Kaggle swag if your question is chosen for discussion during the livestream!

In [1]:
import logging
import os

#Cleraring the logs folder if it exists
for log_files in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_files):
        os.remove(log_files)
        print(f"{log_files} deleted successfully")

# Congigure logging with debug log level
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format='%(asctime)s %(levelname)-8s [%(filename)s:%(lineno)d] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

print("Logging configured successfully")

logger.log deleted successfully
Logging configured successfully


In [2]:
#Checking google api key
import os
import dotenv
dotenv.load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if GOOGLE_API_KEY is None:
    
    print("Please set the GOOGLE_API_KEY environment variable")
else:
    print("Google API key found and loaded successfully!")
    

Google API key found and loaded successfully!


In [3]:
# Creating a research Agent
!adk create research-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in d:\Learning\IITKML\Self\Kaggle\5-Day AI Agents Intensive Course with Google\Day4_13Nov2025\research-agent:
- .env
- __init__.py
- agent.py



In [4]:
%%writefile research-agent/agent.py

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search

from google.genai import types
from typing import List

retry_config = types.HttpRetryoptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)

#Intentionally passing incorrect data
def count_papers(papers: str):
    """
    This function counts the number of papers in a list of strings.
    Args:
      papers: A list of strings, where each string is a research paper.
    Returns:
      The number of papers in the list.
    """
    return len(papers)

#Google Search Agent Tool
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Searches for information using Google search",
    instruction="""Use the google_search tool to find information on the given topic. Return the raw search results.
    If the user asks for a list of papers, then give them the list of research papers you found and not the summary.""",
    tools=[google_search]
)

# Root agent
root_agent = LlmAgent(
    name="research_paper_finder_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Your task is to find research papers and count them. 

    You MUST ALWAYS follow these steps:
    1) Find research papers on the user provided topic using the 'google_search_agent'. 
    2) Then, pass the papers to 'count_papers' tool to count the number of papers returned.
    3) Return both the list of research papers and the total number of papers.
    """,
    tools=[AgentTool(agent=google_search_agent), count_papers]
)

Overwriting research-agent/agent.py


In [ ]:
%%writefile hello.py
print("Hello, world!")


In [9]:
# Need to run the agent first before running this script
# url_prefix = get_adk_proxy_url()
!python agent.py --url_prefix $url_prefix

C:\Users\sande\AppData\Local\Programs\Python\Python311\python.exe: can't open file 'd:\\Learning\\IITKML\\Self\\Kaggle\\5-Day AI Agents Intensive Course with Google\\Day4_13Nov2025\\agent.py': [Errno 2] No such file or directory


In [7]:
!adk web --log_level DEBUG --url_prefix {url_prefix}


+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http://127.0.0.1:8000.                         |
+-----------------------------------------------------------------------------+


+-----------------------------------------------------------------------------+
| ADK Web Server shutting down...                                             |
+-----------------------------------------------------------------------------+



d:\Learning\.venv\Lib\site-packages\google\adk\cli\fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
d:\Learning\.venv\Lib\site-packages\google\adk\auth\credential_service\in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
2025-11-14 18:33:28,461 - DEBUG - selector_events.py:54 - Using selector: SelectSelector
INFO:     Started server process [31352]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): only one usage of each socket address (protocol/network address/por